## RNN using PyTorch

In [ ]:
import pandas as pd

df = pd.read_csv('12_100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
# tokenize
def tokenize(text):
    text = text.lower()
    text = text.replace('?', '')
    text = text.replace("'", "")
    return text.split()

In [ ]:
# vocab
vocab = {"<UNK>":0}

In [ ]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenizedd_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question + tokenizedd_answer

    for token in merged_tokens:
        
        if token not in vocab:
            vocab[token] = len(vocab)


In [ ]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [ ]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [ ]:
len(vocab)

324

In [ ]:
# convert words to numerical indices
def text_to_indices(text, vocab):

    indexed_vocab = []

    for token in tokenize(text):
        if token in vocab:
            indexed_vocab.append(vocab[token])
        else:
            indexed_vocab.append(vocab['<UNK>'])

    return indexed_vocab

In [ ]:
text_to_indices("What is campusX", vocab)

[1, 2, 0]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        numerical_question = text_to_indices(self.df.iloc[idx]['question'], self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[idx]['answer'], self.vocab)

        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [ ]:
dataset = QADataset(df, vocab)

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
for question, answer in dataloader:
    print("Question:", question)
    print("Answer:", answer)
    print("\n")

Question: tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]])
Answer: tensor([[28]])


Question: tensor([[  1,   2,   3, 221,   5, 222, 223, 224]])
Answer: tensor([[225]])


Question: tensor([[ 42, 137,   2, 138,  39, 139]])
Answer: tensor([[53]])


Question: tensor([[ 10,  96,   3, 104, 239]])
Answer: tensor([[240]])


Question: tensor([[ 42, 167,   2,   3,  17, 168, 169]])
Answer: tensor([[170]])


Question: tensor([[  1,   2,   3,   4,   5, 206]])
Answer: tensor([[207]])


Question: tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]])
Answer: tensor([[61]])


Question: tensor([[ 1,  2,  3, 69,  5, 53]])
Answer: tensor([[260]])


Question: tensor([[  1,  87, 229, 230, 231, 232]])
Answer: tensor([[233]])


Question: tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]])
Answer: tensor([[173]])


Question: tensor([[ 1,  2,  3,  4,  5, 99]])
Answer: tensor([[100]])


Question: tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]])
Answer: tensor([[307]])


Question: tensor([[78, 7

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
        self.rnn = nn.RNN(50, 64, batch_first=True)
        self.fc = nn.Linear(64, vocab_size)

    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output

In [ ]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [ ]:
learning_rate = 0.001
epochs = 30

In [ ]:
model = SimpleRNN(len(vocab))
model

SimpleRNN(
  (embedding): Embedding(324, 50)
  (rnn): RNN(50, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=324, bias=True)
)

In [ ]:
criteriom = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop

for epoch in range(epochs):

    total_loss = 0

    for question, answer in dataloader:

        optimizer.zero_grad()

        # forward pass
        output = model(question)

        # loss -> output shape (1, 324) - (1)
        loss = criteriom(output, answer[0])

        # gradient
        loss.backward()

        # update 
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 520.9650
Epoch 2, Loss: 454.8414
Epoch 3, Loss: 377.8403
Epoch 4, Loss: 311.2656
Epoch 5, Loss: 257.5481
Epoch 6, Loss: 208.8780
Epoch 7, Loss: 165.2593
Epoch 8, Loss: 127.5378
Epoch 9, Loss: 97.6881
Epoch 10, Loss: 74.7591
Epoch 11, Loss: 57.9927
Epoch 12, Loss: 45.8129
Epoch 13, Loss: 36.8879
Epoch 14, Loss: 30.0792
Epoch 15, Loss: 24.5247
Epoch 16, Loss: 20.7758
Epoch 17, Loss: 17.5790
Epoch 18, Loss: 14.8766
Epoch 19, Loss: 12.7244
Epoch 20, Loss: 11.1538
Epoch 21, Loss: 9.6910
Epoch 22, Loss: 8.5342
Epoch 23, Loss: 7.5324
Epoch 24, Loss: 6.6981
Epoch 25, Loss: 6.0168
Epoch 26, Loss: 5.4392
Epoch 27, Loss: 4.9296
Epoch 28, Loss: 4.4771
Epoch 29, Loss: 4.0715
Epoch 30, Loss: 3.7270


In [ ]:
def predict(model, question, threshold=0.5):

    # convert question to numbers
    numerical_question = text_to_indices(question, vocab)

    # tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # send to model
    output = model(question_tensor)

    # convert logits into probabilities
    probs = torch.nn.functional.softmax(output, dim=1)

    # find index of max probs
    value, index = torch.max(probs, dim=1)

    if value < threshold:
        print("I don't know")
    else:
        print(list(vocab.keys())[index])

In [ ]:
predict(model, "What is campusx")

I don't know
